<a href="https://colab.research.google.com/github/rocabrera/language-uncertainty/blob/master/finetunning_bucket_custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet datasets transformers sentencepiece

     |████████████████████████████████| 362 kB 7.3 MB/s 
     |████████████████████████████████| 4.4 MB 64.4 MB/s 
     |████████████████████████████████| 1.2 MB 67.3 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 212 kB 70.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.7 MB/s 
     |████████████████████████████████| 140 kB 17.1 MB/s 
     |████████████████████████████████| 596 kB 21.3 MB/s 
     |████████████████████████████████| 127 kB 53.9 MB/s 
     |████████████████████████████████| 6.6 MB 43.3 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 73.6 MB/s 
     |████████████████████████████████| 271 kB 77.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which

In [2]:
!nvidia-smi

Tue Jul 19 12:02:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict
from transformers import (T5Tokenizer, 
                          T5ForConditionalGeneration,
                          Seq2SeqTrainer, 
                          Seq2SeqTrainingArguments)
from tqdm.notebook import tqdm

In [4]:
def squad_tokenizer_prompt(tokenizer, sample: dict):
    """
    Importante notar que não podemos retornar tensores se quisermos fazer em batch!
    """
    
    if isinstance(sample["question"], list):
        encoding = tokenizer(
            ['question: ' + q for q in sample["question"]],
            ['context: ' + c for c in sample["context"]],
            max_length=396,
            padding="max_length",
            truncation="only_second", # Se nao me engano trunca somente o contexto .... Problematico dependendo de onde a resposta esta
            return_attention_mask=True,
            add_special_tokens=True,
        )
                
    else:
        encoding = tokenizer(
            'question: ' + sample["question"],
            'context: ' + sample["context"],
            max_length=396,
            padding="max_length",
            truncation="only_second", # Se nao me engano trunca somente o contexto .... Problematico dependendo de onde a resposta esta
            return_attention_mask=True,
            add_special_tokens=True,
        )
    
    return encoding


def squad_tokenizer_answer(tokenizer, sample: dict):

    "Talvez mudar depois"

    if isinstance(sample["answers_bucket_uncertainty"], list):
        
        """
        Utilizei o eval porque estava lendo o dicionario do csv como string
        """

        texts = [eval(s)["text"][0] for s in sample["answers_bucket_uncertainty"]] # Tokenizando a primeira resposta *
        answer_encoding = tokenizer(
            texts,
            max_length=32,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )
                
    else:
        answer_encoding = tokenizer(
        sample["answers_bucket_uncertainty"]["text"][0],  # Tokenizando a primeira resposta *
        max_length=32,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )
    labels_ids = answer_encoding["input_ids"]
    # https://huggingface.co/docs/transformers/model_doc/t5
    labels_ids[labels_ids == tokenizer.pad_token_id] = -100
    encoding = {"label": labels_ids.tolist(), "first_answer": texts}

    return encoding

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# **Loading data**

In [6]:
# No futuro seria legal fazer um mount via código... não consegui.. talvez esteja relacionado 
train = load_dataset("csv", data_files='/content/drive/MyDrive/UNICAMP/squadshifts_aggregated/squadshifts_aggregated_train.csv')["train"]
valid = load_dataset("csv", data_files='/content/drive/MyDrive/UNICAMP/squadshifts_aggregated/squadshifts_aggregated_eval.csv')["train"]
test = load_dataset("csv", data_files='/content/drive/MyDrive/UNICAMP/squadshifts_aggregated/squadshifts_aggregated_test.csv')["train"]


Using custom data configuration default-13638a956acc048a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-13638a956acc048a/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Using custom data configuration default-c3419c83822c9615


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c3419c83822c9615/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
print(len(train))
print(len(valid))
print(len(test))

6712
642
584


# **Load Model**

In [8]:
# One can use T5ForConditionalGeneration (or the Tensorflow/Flax variant), which includes the language modeling head on top of the decoder.
MODEL_NAME = "t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

# **Preprocessing**

In [9]:
def preprocessing(data: Dataset):

  return (
      data.map(lambda x: squad_tokenizer_prompt(tokenizer, x), batched=True, num_proc=8)
          .map(lambda x: squad_tokenizer_answer(tokenizer, x), batched=True, num_proc=8)
          .remove_columns("answers")
  )

train = preprocessing(train)
valid = preprocessing(valid)
test = preprocessing(test)

Parameter 'function'=<function preprocessing.<locals>.<lambda> at 0x7feb01a94170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

# **Training Model**

In [10]:
# Ref1: https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html
# Ref2: https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html#:~:text=There%20are%20two%20dominant%20metrics,possible%20correct%20answers%20is%20computed.
def compute_exact_match(predict_text: str, label_text:str):
  return int(predict_text == label_text)


def compute_f1(predict_text: str, label_text:str):
    pred_tokens = predict_text.split()
    truth_tokens = label_text.split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(eval_preds):
  token_ids, labels_ids = eval_preds
  decoded_texts = tokenizer.batch_decode(
                token_ids,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False)
  
  aux = labels_ids.copy()
  aux[aux == -100] = tokenizer.pad_token_id
  label_texts = tokenizer.batch_decode(
                aux,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False)
  
  f1, em = zip(*[(compute_f1(decoded_text, label_text),compute_exact_match(decoded_text, label_text))
                 for decoded_text, label_text in zip(decoded_texts, label_texts)])
  # bleu = sacrebleu.corpus_bleu(decoded_texts, label_texts)

  return {"EM": np.mean(em), "F1": np.mean(f1)}

In [11]:
"""
Obs: predict_with_generate é importante para não nos preocuparmos com o decode do output do T5

Quando usamos esse parâmetro setado para true o argumento eval_preds na funcao compute_metrics 
recebe os tokens já... 
"""

batch_size = 16
num_epochs = 20
# logging_steps = train.num_rows // batch_size
model_name = f"{MODEL_NAME}-squadshifts_aggregated"
training_args = Seq2SeqTrainingArguments(output_dir=model_name,
                                  num_train_epochs=num_epochs,
                                  learning_rate=2e-5, # Existe boas praticas para esse número no caso especifico do T5
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=30,
                                  log_level="error",
                                  predict_with_generate=True)

In [12]:
# Ref: https://huggingface.co/docs/transformers/main_classes/trainer

trainer = Seq2SeqTrainer(model=model, 
                         args=training_args,
                         compute_metrics=compute_metrics,
                         train_dataset=train,
                         eval_dataset=valid,
                         tokenizer=tokenizer)

In [13]:
def evaluate(trainer: Seq2SeqTrainer, dataset: Dataset, device):

  ems = []
  f1s = []
  predict_texts = []

  for sample in tqdm(dataset):
    token_ids = trainer.model.generate(input_ids=torch.as_tensor(sample["input_ids"]).reshape(1,-1).to(device))

    predict_text = tokenizer.batch_decode(token_ids,
                                          skip_special_tokens=True,
                                          clean_up_tokenization_spaces=False)
    
    f1s.append(compute_f1(predict_text[0], sample["first_answer"]))
    ems.append(compute_exact_match(predict_text[0], sample["first_answer"]))
    predict_texts.append(predict_text[0])

    
  return ems, f1s, predict_texts

In [14]:
ems, f1s, prediction_before_finetunning = evaluate(trainer, test, device)
test = test.add_column("predict_before_finetunning", prediction_before_finetunning)
print("ANTES DO FINETUNNING")
print("Exact Match:")
print(np.mean(ems))
print("F1:")
print(np.mean(f1s))

  0%|          | 0/584 [00:00<?, ?it/s]

ANTES DO FINETUNNING
Exact Match:
0.0
F1:
0.46232743486119665


In [15]:
ems, f1s, prediction_before_finetunning = evaluate(trainer, train, device)
train = train.add_column("predict_before_finetunning", prediction_before_finetunning)
print("ANTES DO FINETUNNING")
print("Exact Match:")
print(np.mean(ems))
print("F1:")
print(np.mean(f1s))

  0%|          | 0/6712 [00:00<?, ?it/s]

ANTES DO FINETUNNING
Exact Match:
0.0
F1:
0.49077799625374113


In [16]:
ems, f1s, prediction_before_finetunning = evaluate(trainer, valid, device)
valid = valid.add_column("predict_before_finetunning", prediction_before_finetunning)
print("ANTES DO FINETUNNING")
print("Exact Match:")
print(np.mean(ems))
print("F1:")
print(np.mean(f1s))

  0%|          | 0/642 [00:00<?, ?it/s]

ANTES DO FINETUNNING
Exact Match:
0.0
F1:
0.46901727438834606


In [17]:
trainer.train();

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Em,F1
1,0.334700,0.342099,0.278816,0.717291
2,0.298600,0.332263,0.281931,0.719290
3,0.251500,0.319198,0.271028,0.708607
4,0.246600,0.324603,0.280374,0.715487
5,0.225100,0.328077,0.283489,0.718029
6,0.212200,0.330980,0.277259,0.714651
7,0.205000,0.331597,0.278816,0.710316
8,0.184300,0.341214,0.278816,0.717220
9,0.174400,0.338138,0.277259,0.718095
10,0.165900,0.348252,0.275701,0.715132


In [18]:
ems, f1s, prediction_after_finetunning = evaluate(trainer, test, device)
test = test.add_column("predict_after_finetunning", prediction_after_finetunning)
print("DEPOIS DO FINETUNNING")
print("Exact Match:")
print(np.mean(ems))
print("F1:")
print(np.mean(f1s))

  0%|          | 0/584 [00:00<?, ?it/s]

DEPOIS DO FINETUNNING
Exact Match:
0.3373287671232877
F1:
0.7324137423062725


In [19]:
ems, f1s, prediction_after_finetunning = evaluate(trainer, train, device)
train = train.add_column("predict_after_finetunning", prediction_after_finetunning)
print("DEPOIS DO FINETUNNING")
print("Exact Match:")
print(np.mean(ems))
print("F1:")
print(np.mean(f1s))

  0%|          | 0/6712 [00:00<?, ?it/s]

DEPOIS DO FINETUNNING
Exact Match:
0.5065554231227652
F1:
0.8686950371268474


In [20]:
ems, f1s, prediction_after_finetunning = evaluate(trainer, valid, device)
valid = valid.add_column("predict_after_finetunning", prediction_after_finetunning)
print("DEPOIS DO FINETUNNING")
print("Exact Match:")
print(np.mean(ems))
print("F1:")
print(np.mean(f1s))

  0%|          | 0/642 [00:00<?, ?it/s]

DEPOIS DO FINETUNNING
Exact Match:
0.2834890965732087
F1:
0.7174649835553707


# **Exporting Results**

In [ ]:
trainer.state.log_history

In [21]:
test.to_csv("/content/drive/MyDrive/UNICAMP/squadshifts_aggregated_bucket_with_predictions_test.csv", index=False)
train.to_csv("/content/drive/MyDrive/UNICAMP/squadshifts_aggregated_bucket_with_predictions_train.csv", index=False)
valid.to_csv("/content/drive/MyDrive/UNICAMP/squadshifts_aggregated_bucket_with_predictions_eval.csv", index=False)


Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

3035108